#### Import Libraries

In [1]:
import pandas as pd
import numpy as np

#### Loading the data

In [2]:
nf_data = pd.read_csv(r"C:\Users\shaik\OneDrive\Desktop\SQL\Netflix_ETL\Netflix_data.csv")
nf_data

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,s1,Movie,Dick Johnson Is Dead,Kirsten Johnson,NaN,United States,"September 25, 2021",2020,PG-13,90 min,Documentaries,"As her father nears the end of his life, filmm..."
1,s2,TV Show,Blood & Water,NaN,"Ama Qamata, Khosi Ngema, Gail Mabalane, Thaban...",South Africa,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, TV Dramas, TV Mysteries","After crossing paths at a party, a Cape Town t..."
2,s3,TV Show,Ganglands,Julien Leclercq,"Sami Bouajila, Tracy Gotoas, Samuel Jouy, Nabi...",NaN,"September 24, 2021",2021,TV-MA,1 Season,"Crime TV Shows, International TV Shows, TV Act...",To protect his family from a powerful drug lor...
3,s4,TV Show,Jailbirds New Orleans,NaN,NaN,NaN,"September 24, 2021",2021,TV-MA,1 Season,"Docuseries, Reality TV","Feuds, flirtations and toilet talk go down amo..."
4,s5,TV Show,Kota Factory,NaN,"Mayur More, Jitendra Kumar, Ranjan Raj, Alam K...",India,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, Romantic TV Shows, TV ...",In a city of coaching centers known to train I...
...,...,...,...,...,...,...,...,...,...,...,...,...
8802,s8803,Movie,Zodiac,David Fincher,"Mark Ruffalo, Jake Gyllenhaal, Robert Downey J...",United States,"November 20, 2019",2007,R,158 min,"Cult Movies, Dramas, Thrillers","A political cartoonist, a crime reporter and a..."
8803,s8804,TV Show,Zombie Dumb,NaN,NaN,NaN,"July 1, 2019",2018,TV-Y7,2 Seasons,"Kids' TV, Korean TV Shows, TV Comedies","While living alone in a spooky town, a young g..."
8804,s8805,Movie,Zombieland,Ruben Fleischer,"Jesse Eisenberg, Woody Harrelson, Emma Stone, ...",United States,"November 1, 2019",2009,R,88 min,"Comedies, Horror Movies",Looking to survive in a world taken over by zo...
8805,s8806,Movie,Zoom,Peter Hewitt,"Tim Allen, Courteney Cox, Chevy Chase, Kate Ma...",United States,"January 11, 2020",2006,PG,88 min,"Children & Family Movies, Comedies","Dragged from civilian life, a former superhero..."


In [3]:
nf_data.shape

(8807, 12)

In [4]:
nf_data.isnull().sum()

show_id            0
type               0
title              0
director        2634
cast             825
country          831
date_added        10
release_year       0
rating             4
duration           3
listed_in          0
description        0
dtype: int64

In [5]:
for col in ['director', 'cast', 'country', 'rating']:
    nf_data[col] = nf_data[col].fillna('Unknown')

In [6]:
nf_data.dropna(inplace=True)

In [7]:
nf_data = nf_data.drop_duplicates()
nf_data.shape

(8794, 12)

In [8]:
for col in ['title', 'director', 'cast', 'country', 'rating', 'listed_in', 'description']:
    nf_data[col] = nf_data[col].astype(str).str.strip()

In [9]:
nf_data['date_added'] = pd.to_datetime(nf_data['date_added'], errors='coerce')
nf_data['year_added'] = nf_data['date_added'].dt.year
nf_data['month_added'] = nf_data['date_added'].dt.month

In [10]:
nf_data['duration_num'] = nf_data['duration'].str.extract('(\d+)').astype('float')
nf_data['duration_type'] = nf_data['duration'].str.extract('([A-Za-z]+)')

In [11]:
nf_data['release_year'] = pd.to_numeric(nf_data['release_year'], errors='coerce').astype('Int64')

In [12]:
nf_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 8794 entries, 0 to 8806
Data columns (total 16 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   show_id        8794 non-null   object        
 1   type           8794 non-null   object        
 2   title          8794 non-null   object        
 3   director       8794 non-null   object        
 4   cast           8794 non-null   object        
 5   country        8794 non-null   object        
 6   date_added     8706 non-null   datetime64[ns]
 7   release_year   8794 non-null   Int64         
 8   rating         8794 non-null   object        
 9   duration       8794 non-null   object        
 10  listed_in      8794 non-null   object        
 11  description    8794 non-null   object        
 12  year_added     8706 non-null   float64       
 13  month_added    8706 non-null   float64       
 14  duration_num   8794 non-null   float64       
 15  duration_type  8794 non-nu

In [13]:
nf_data['genres_list'] = nf_data['listed_in'].apply(lambda x: [g.strip() for g in x.split(',')])

In [14]:
genres_exploded = nf_data[['show_id', 'genres_list']].explode('genres_list')
genres_exploded['genres_list'] = genres_exploded['genres_list'].str.strip()

In [15]:
unique_genres = pd.DataFrame(genres_exploded['genres_list'].unique(), columns=['genre_name'])
unique_genres['genre_id'] = unique_genres.index + 1
show_genre_mapping = genres_exploded.merge(unique_genres, left_on='genres_list', right_on='genre_name', how='left')
show_genre_mapping_final = show_genre_mapping[['show_id', 'genre_id']]


In [16]:
nf_data['cast_list'] = nf_data['cast'].apply(lambda x: [actor.strip() for actor in x.split(',')])
cast_exploded = nf_data[['show_id', 'cast_list']].explode('cast_list')
unique_actors = pd.DataFrame(cast_exploded['cast_list'].unique(), columns=['actor_name'])
unique_actors['actor_id'] = unique_actors.index + 1
show_actor_mapping = cast_exploded.merge(unique_actors, left_on='cast_list', right_on='actor_name', how='left')
show_actor_mapping_final = show_actor_mapping[['show_id', 'actor_id']]


In [17]:
nf_data['director_list'] = nf_data['director'].apply(lambda x: [d.strip() for d in x.split(',')])
director_exploded = nf_data[['show_id', 'director_list']].explode('director_list')
unique_directors = pd.DataFrame(director_exploded['director_list'].unique(), columns=['director_name'])
unique_directors['director_id'] = unique_directors.index + 1
show_director_mapping = director_exploded.merge(unique_directors, left_on='director_list', right_on='director_name', how='left')
show_director_mapping_final = show_director_mapping[['show_id', 'director_id']]

In [18]:
nf_data.drop(['listed_in', 'cast', 'genres_list', 'cast_list', 'director_list', 'director', 'description', 'duration'], axis=1, inplace=True)
nf_data.to_csv("netflix_cleaned.csv", index=False)

In [19]:
unique_genres.to_csv('genres_table.csv', index=False)
show_genre_mapping_final.to_csv('show_genre_mapping.csv', index=False)

unique_actors.to_csv('actors_table.csv', index=False)
show_actor_mapping_final.to_csv('show_actor_mapping.csv', index=False)

unique_directors.to_csv('directors_table.csv', index=False)
show_director_mapping_final.to_csv('show_director_mapping.csv', index=False)